توضیحات بیشتر در فایل پی دی اف توضیحات

In [1]:
import os
import cv2
import numpy as np
import math
from skimage.segmentation import mark_boundaries


ModuleNotFoundError: No module named 'skimage'

آماده سازی تصویر در سه فضای آرجی بی، لب و خاکستری آماده سازی و گرادیان

In [ ]:

slic = cv2.imread( os.getcwd()+'\\'+'slic.jpg',cv2.IMREAD_UNCHANGED)
slic_bw = cv2.imread( os.getcwd()+'\\'+'slic.jpg',cv2.IMREAD_GRAYSCALE)
slic_lab = cv2.cvtColor(slic,cv2.COLOR_BGR2LAB) 

horz_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,1,1],[0,0,0],[-1,-1,-1]]))  
vert_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,0,-1],[1,0,-1],[1,0,-1]]))
gradian = np.sqrt(horz_gradian**2 + vert_gradian**2)

تابع تولید کننده مراکز اولیه بطور یکنواخت

In [ ]:
def generate_centers(delta_rows,r ,delta_cols, c):
    par = 20
    sotun = np.linspace(delta_rows-par, min(delta_rows*(r), slic_bw.shape[0])-par,num = r)
    satr = np.linspace(delta_cols-par, min(delta_cols*(c), slic_bw.shape[1])-par,num = c)
    #center is a 2D thing 
    centers_x,centers_y = np.meshgrid(satr, sotun)  
    centers = np.vstack([np.ravel(centers_x), np.ravel(centers_y)]).T
    centers = centers.astype(int)
    return centers

تابع حرکت دهنده ی اولیه ی نقاط در همسایگی ده در ده

In [ ]:
def first_move_centers(centers, new_cluster_center):
    pan = 5
    #mishe 10x10
    i=0
    for cluster_center in centers:
        coor_x = cluster_center[0]
        coor_y = cluster_center[1]

        partition = gradian[coor_y-pan:coor_y+pan+1, coor_x-pan:coor_x+pan+1]
        argmin = np.unravel_index(partition.argmin(), partition.shape)
        new_cluster_center[i] = [coor_x + argmin[1] - pan, coor_y + argmin[0] - pan ]
        i+=1
    return new_cluster_center

تابع اصلی انجام دهنده الگوریتم اسلیک

In [ ]:
def slic_with_K(img_lab,r,c,alpha, threshold, stop_itr):
    global gradian
    global slic_bw
 
    img = cv2.cvtColor(img_lab, cv2.COLOR_LAB2BGR)
    delta_rows = slic_bw.shape[0]//r
    delta_cols = slic_bw.shape[1]//c
    labels = np.zeros(slic_bw.shape, dtype = 'uint8')
    label_distances = np.full(slic_bw.shape, np.inf, dtype = 'float32')
    
    centers = generate_centers(delta_rows,r, delta_cols, c)   
    new_cluster_center = np.zeros(centers.shape, dtype = 'uint16')
    prev_centers = np.zeros(centers.shape, dtype = 'uint16')

    new_cluster_center[:,:] = first_move_centers(centers, new_cluster_center)
       
    centers[:,:] = new_cluster_center[:,:]
    prev_centers[:,:] = centers[:,:]
    
    distance_window = make_distance_window(delta_rows, delta_cols)
    
    # print(distance_window.shape)
    # print(points)
        
    t = delta_cols
    delta_cols = delta_rows
    delta_rows = t
    counter = 0 
    
    #for similarity difference with center
    threshshold = 300
    #for stoping outer loop
    stop_threshold = 10
    # stop_itr = 1
    itr_cnt = 0
    # while (itr_cnt > 0 and np.mean(np.abs(np.sum(centers - prev_centers))) > threshshold) or itr_cnt > stop_itr or itr_cnt==0 :
    while itr_cnt < stop_itr:
        counter = 0 
        itr_cnt +=1
        for cluster_center in centers:
                counter+=1
                coor_x = cluster_center[0]
                coor_y = cluster_center[1]
                startx1 = max(coor_x - delta_rows, 0)
                endx1 = min(coor_x + delta_rows, slic_bw.shape[1]-1)
                starty1= max(coor_y - delta_cols,0)
                endy1 = min(coor_y + delta_cols, slic_bw.shape[0]-1)
            
                partition_lab = slic_lab[starty1:endy1+1,startx1:endx1+1,:]
                
                
                startx2 = delta_rows - coor_x + startx1
                endx2 = delta_rows - coor_x + endx1
                starty2 = delta_cols - coor_y + starty1
                endy2 = delta_cols - coor_y + endy1       
                
                partition_dist = distance_window[starty2:endy2+1,startx2:endx2+1]
            
                dist_matrix = dist(partition_lab, partition_dist, slic_lab[coor_y][coor_x], alpha)
                # print(dist_matrix)
                # print(np.where(inted_boolean_matrix>0))
                updated_cluster_center = matrix_corrector(dist_matrix, cluster_center, counter,startx1, endx1, starty1, endy1, labels, label_distances, threshold)
                #  = update_cluster_center()
                new_cluster_center[counter-1] = updated_cluster_center
        prev_centers[:,:] = centers[:,:]
        centers[:,:] = new_cluster_center[:,:]
        # print(itr_cnt)


    global show_labels
    show_labels = labels[:,:]

    return image_process(img, labels)

    

تابع تولید کننده جدولی که در هر درایه ی آن، فاصله اقلیدسی آن درایه تا مرکز جدول قرار گرفته

In [ ]:
def make_distance_window(delta_rows, delta_cols):
    satr = np.arange(-delta_rows, +delta_rows+1)
    satr = np.tile(satr, (2*delta_cols+1, 1)).T
    sotun = np.arange(-delta_cols, delta_cols+1)
    sotun = np.tile(sotun, (2*delta_rows+1,1))
    sotun = sotun**2
    satr = satr**2
    return np.sqrt(satr + sotun)

تابع محاسبه فاصله کلی از روی فاصله در فضای لب و فاصله اقلیدسی 

In [ ]:
def dist(lab, dist, cnt_lab, alpha):
    d_lab = np.sqrt((lab[:,:,0] -  cnt_lab[0])**2 + (lab[:,:,1] -  cnt_lab[1])**2 + (lab[:,:,2] -  cnt_lab[2])**2)
    # alpha = 0.4
    return d_lab + alpha*dist

تابع آپدیت ماتریس های متناظر با لیبل هر پیکسل و فاصله هر پیکسل تا کلاستر سنتر متناظر با آن در هر ایتریشن

In [ ]:
def matrix_corrector(dist_matrix, center, k, sx, ex, sy, ey, labels, label_distances, threshold):
 
    
    partition_dist = label_distances[sy:ey+1,sx:ex+1]
    partition_labels = labels[sy:ey+1,sx:ex+1]
    
    acc = (dist_matrix < threshold) & (dist_matrix < partition_dist)
    partition_dist[acc] = dist_matrix[acc]
    partition_labels[acc] = k 

    k_labeled_points_bool = np.equal(labels, k).astype(int)

    args = np.argwhere(k_labeled_points_bool)
    mean_x = np.mean(args[:,0])
    mean_y = np.mean(args[:,1])
 
    return np.array([mean_y, mean_x]).astype(int)

تابعی که در نقاط مرزی کلاستر ها نقاط قرمز رسم می کند

In [ ]:
def image_process(img, labels):
    
    kernel = np.ones((5,5),np.uint8)
    # labels = cv2.morphologyEx(labels, cv2.MORPH_GRADIENT, kernel)
    labels = cv2.erode(labels,kernel,iterations = 1)

    edges_bool = cv2.Canny(labels,1,1)

    for edge in np.argwhere(edges_bool):
        img = cv2.circle(img, (edge[1],edge[0]), radius=1, color=(0, 0, 255), thickness=-1)


    
    # cv2.imshow('img', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img.astype(int)

اجرای اسلیک با ۶۴ نقطه مرکزی

In [ ]:
# img_res = slic_with_K(slic_lab,8,8,0.2, 750, 7)
# cv2.imwrite('res05.jpg',img_res)

اجرای اسلیک با ۲۵۶ نقطه مرکزی

In [ ]:
slic = cv2.imread( os.getcwd()+'\\'+'slic.jpg',cv2.IMREAD_UNCHANGED)

slic = cv2.resize(slic, (slic.shape[1]//2, slic.shape[0]//2))
slic_bw = cv2.cvtColor(slic, cv2.COLOR_BGR2GRAY)
slic_lab = cv2.cvtColor(slic,cv2.COLOR_BGR2LAB) 
horz_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,1,1],[0,0,0],[-1,-1,-1]]))  
vert_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,0,-1],[1,0,-1],[1,0,-1]]))
gradian = np.sqrt(horz_gradian**2 + vert_gradian**2)

img_res = slic_with_K(slic_lab,16,16,0.4, 600, 6)
cv2.imwrite('res06.jpg',img_res)

اجرای اسلیک با ۱۰۲۴ نقطه مرکزی

In [ ]:
# slic = cv2.imread( os.getcwd()+'\\'+'slic.jpg',cv2.IMREAD_UNCHANGED)

# slic = cv2.resize(slic, (slic.shape[1]//2, slic.shape[0]//2))
# slic_bw = cv2.cvtColor(slic, cv2.COLOR_BGR2GRAY)
# slic_lab = cv2.cvtColor(slic,cv2.COLOR_BGR2LAB) 
# horz_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,1,1],[0,0,0],[-1,-1,-1]]))  
# vert_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,0,-1],[1,0,-1],[1,0,-1]]))
# gradian = np.sqrt(horz_gradian**2 + vert_gradian**2)

# img_res = slic_with_K(slic_lab,32,32,0.4, 100, 4)
# cv2.imwrite('res07.jpg',img_res)

اجرای اسلیک با ۲۰۴۸ نقطه مرکزی

In [ ]:
# slic = cv2.imread( os.getcwd()+'\\'+'slic.jpg',cv2.IMREAD_UNCHANGED)

# slic = cv2.resize(slic, (slic.shape[1]//2, slic.shape[0]//2))
# slic_bw = cv2.cvtColor(slic, cv2.COLOR_BGR2GRAY)
# slic_lab = cv2.cvtColor(slic,cv2.COLOR_BGR2LAB) 
# horz_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,1,1],[0,0,0],[-1,-1,-1]]))  
# vert_gradian = cv2.filter2D(slic_bw,-1,np.float32([[1,0,-1],[1,0,-1],[1,0,-1]]))
# gradian = np.sqrt(horz_gradian**2 + vert_gradian**2)

# img_res = slic_with_K(slic_lab,32,64,0.55, 100, 4)
# cv2.imwrite('res08.jpg',img_res)

True